In [1]:
%load_ext autoreload
%autoreload 2
import pandas
pandas.set_option('display.max_rows', None)

In [ ]:
from absbox import API,EnginePath
#localAPI = API("http://localhost:8081",check=False)
localAPI = API(EnginePath.DEV,check=False)

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.28.7, server:0.28.18

### Revolving Feature Example

In [3]:
revol_asset1 = ["Mortgage"
                ,{"originBalance":220,"originRate":["fix",0.045],"originTerm":30
                  ,"freq":"Monthly","type":"Level","originDate":"2021-07-01"}
                  ,{"currentBalance":220
                  ,"currentRate":0.08
                  ,"remainTerm":6
                  ,"status":"current"}]

revol_asset2 = ["Mortgage"
                ,{"originBalance":220,"originRate":["fix",0.045],"originTerm":30
                  ,"freq":"Monthly","type":"Level","originDate":"2021-07-01"}
                  ,{"currentBalance":220
                  ,"currentRate":0.08
                  ,"remainTerm":12
                  ,"status":"current"}]


revolvingPool1 = (["constant",revol_asset1]
                 ,("Pool",("Mortgage",{"CDR":0.0},None,None,None)
                   ,None
                   ,None))

revolvingPool2 = (["constant",revol_asset2]
                 ,("Pool",("Mortgage",{"CDR":0.1},None,None,None)
                   ,None
                   ,None))

#### Single revolving pool, Single Deal Pool

* A deal with a *Single* pool and there is only *One* revolving pool
* Use `buyAsset` waterfall action

In [ ]:
from absbox.local.generic import Generic

test00 = Generic(
    "TEST01"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":24
          ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":1000
             ,"rate":0.07
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30},"feeStart":"2021-06-15"}),)
    ,{"amortizing":[
         ["payFee","acc01",['trusteeFee']]
         ,["payInt","acc01",["A1"]]
         ,["If",["=","2022-08-20"]
            ,["buyAsset",["Current|Defaulted",1.0,0],"acc01"]
            ]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payPrinResidual","acc01",["B"]]
     ]}
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
)


r = localAPI.run(test00
                 ,runAssump=[("revolving",*revolvingPool2)]
                 ,read=True)

r['pool']['flow']['PoolConsol'].loc["2022-07-31":"2022-11-30"]

Warning Message from server:
Account acc01 has cash to be distributed

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2022-07-31,1235.10,90.66,8.83,0,0.00,0,0.00,0.08,None,None,964.90,0,0,0.00,0,0.00
2022-08-31,1243.33,91.26,8.23,0,0.00,0,0.00,0.08,None,None,1056.16,0,0,0.00,0,0.00
2022-09-30,1142.65,99.80,8.27,0,0.88,0,0.88,0.08,None,None,1155.96,0,0,0.88,0,0.88
2022-10-31,1041.47,100.40,7.60,0,0.78,0,0.78,0.08,None,None,1256.36,0,0,1.66,0,1.66
2022-11-30,939.76,100.98,6.93,0,0.73,0,0.73,0.08,None,None,1357.34,0,0,2.39,0,2.39


#### Single revolving pool, Multiple Deal Pool

* The deal has multiple pools ( which model as a map)
* Use `buyAssetFrom` waterfall action
    * ` ["buyAsset2", <Pricing Method>, <Source Account>, <Limit>, <Source of Revloving Pool>, <Target Pool>] `

In [ ]:
test00_A = Generic(
    "TEST01"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{"PoolA":{'assets':[["Mortgage"
                  ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
                    ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
                    ,{"currentBalance":2200
                    ,"currentRate":0.08
                    ,"remainTerm":30
                    ,"status":"current"}]]}
       ,"PoolB":{'assets':[["Loan"
                        ,{"originBalance": 80000
                          ,"originRate": ["fix",0.08]
                          ,"originTerm": 36
                          ,"freq": "Monthly"
                          ,"type": "i_p"
                          ,"originDate": "2021-02-01"}
                        ,{"currentBalance": 65000
                          ,"currentRate": 0.06
                          ,"remainTerm": 30
                          ,"status": "Current"}]]}
      }
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":1000
             ,"rate":0.07
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30},"feeStart":"2021-06-15"}),)
    ,{"amortizing":[
         ["payFee","acc01",['trusteeFee']]
         ,["payInt","acc01",["A1"]]
         ,["If",["=","2022-08-20"]
            ,["buyAsset2",["Current|Defaulted",1.0,0],"acc01",None,None,"PoolA"]
            ]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payPrinResidual","acc01",["B"]]
     ]}
    ,[[["PoolA"],"CollectedCash","acc01"]
     ,[["PoolB"],"CollectedCash","acc02"]]
    ,None
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
)

r = localAPI.run(test00_A
           ,runAssump=[("revolving",*revolvingPool2)]
           ,read=True)

r['pool']['flow']['PoolA'].loc["2022-07-31":"2022-11-30"]

Warning Message from server:
Account acc01 has cash to be distributed

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2022-07-31,1007.32,73.95,7.20,0,0.00,0,0.00,0.08,None,None,1192.68,0,0,0.00,0,0.00
2022-08-31,1014.04,74.43,6.71,0,0.00,0,0.00,0.08,None,None,1267.11,0,0,0.00,0,0.00
2022-09-30,931.92,81.40,6.74,0,0.72,0,0.72,0.08,None,None,1348.51,0,0,0.72,0,0.72
2022-10-31,849.41,81.88,6.19,0,0.63,0,0.63,0.08,None,None,1430.39,0,0,1.35,0,1.35
2022-11-30,766.46,82.36,5.65,0,0.59,0,0.59,0.08,None,None,1512.75,0,0,1.94,0,1.94


In [15]:
r['pool']['flow']['PoolB'].loc["2022-07-31":"2022-11-30"]

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,
2022-07-31,65000,0,324.93,0,0,0,0,0.06,0,0,0,0,0,0
2022-08-31,65000,0,335.79,0,0,0,0,0.06,0,0,0,0,0,0
2022-09-30,65000,0,335.79,0,0,0,0,0.06,0,0,0,0,0,0
2022-10-31,65000,0,324.93,0,0,0,0,0.06,0,0,0,0,0,0
2022-11-30,65000,0,335.79,0,0,0,0,0.06,0,0,0,0,0,0


#### Multiple revolving pool, Multiple Deal Pool

A deal transaction may have multiple pools which doesn't share same asset type. In this case, the `PoolA` has a list of `Mortgage` while `PoolB` has a list of `Loan`.

In the `buyAsset2` action ,user can specify the :
* `source` revolving pool from the assumption where the buy action perform from.
* `target` pool which the new bought asset belong to

In [ ]:
test02 = Generic(
    "TEST01"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{"PoolA":{'assets':[["Mortgage"
                  ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
                    ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
                    ,{"currentBalance":2200
                    ,"currentRate":0.08
                    ,"remainTerm":30
                    ,"status":"current"}]]}
       ,"PoolB":{'assets':[["Loan"
                        ,{"originBalance": 80000
                          ,"originRate": ["fix",0.08]
                          ,"originTerm": 36
                          ,"freq": "Monthly"
                          ,"type": "i_p"
                          ,"originDate": "2021-02-01"}
                        ,{"currentBalance": 65000
                          ,"currentRate": 0.06
                          ,"remainTerm": 30
                          ,"status": "Current"}]
                          ]
                }
      }
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":1000
             ,"rate":0.07
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30},"feeStart":"2021-06-15"}),)
    ,{"amortizing":[
         ["payFee","acc01",['trusteeFee']]
         ,["payInt","acc01",["A1"]]
         ,["If",["=","2022-08-20"]
            ,["buyAsset2",["Current|Defaulted",1.0,0],"acc01",None,"Pool3","PoolB"]
            ]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payPrinResidual","acc01",["B"]]
     ]}
    ,[[["PoolA"],"CollectedCash","acc01"]
     ,[["PoolB"],"CollectedCash","acc02"]]
    ,None
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
)


revol_asset3 =["Loan"
                ,{"originBalance": 80000
                  ,"originRate": ["fix",0.08]
                  ,"originTerm": 36
                  ,"freq": "Monthly"
                  ,"type": "i_p"
                  ,"originDate": "2021-02-01"}
                ,{"currentBalance": 65000
                  ,"currentRate": 0.06
                  ,"remainTerm": 30
                  ,"status": "Current"}]


revolvingPool3 = (["constant",revol_asset3]
                 ,("Pool",("Loan",{"CDR":0.1},None,None,None)
                   ,None
                   ,None))

r = localAPI.run(test02
         ,runAssump=[("revolving"
                       ,{"Pool1":revolvingPool1,"Pool3":revolvingPool3})]
         ,read=True)

r['pool']['flow']['PoolB'].loc["2022-07-31":"2022-11-30"]

Warning Message from server:
Account acc01 has cash to be distributed

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,
2022-07-31,65000.00,0.0,324.93,0,0.00,0,0.00,0.06,0.0,0,0,0.00,0,0.00
2022-08-31,65081.15,0.0,335.79,0,0.00,0,0.00,0.06,0.0,0,0,0.00,0,0.00
2022-09-30,65080.43,0.0,336.19,0,0.72,0,0.72,0.06,0.0,0,0,0.72,0,0.72
2022-10-31,65079.74,0.0,325.32,0,0.69,0,0.69,0.06,0.0,0,0,1.41,0,1.41
2022-11-30,65079.03,0.0,336.19,0,0.71,0,0.71,0.06,0.0,0,0,2.12,0,2.12
